## ccdproc-08: Photometry of stars 

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.
We are also using ``ccdproc`` package.

In [ ]:
from astropy.io import fits
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ccdproc import CCDData, Combiner

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=12)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=12)
    return media, std

### Opening the FITS files to be combined

**List of files to be combined**

These are three consecutive images of NGC4941 in R band taken with ALFOSC at NOT 2008

In [ ]:
#directory='NOT_2008_N1/'
directory='N1/'
files = ['120077','120078','120079']       # fzt_ALrd+files[i]

Opening the FITS files and putting the data into a list of numpy 2-D arrays.

In [ ]:
image = []
for i in range(len(files)):
    image.append(fits.open(directory+'fzt_ALrd'+str(files[i])+'.fits')[0])
for i in range(len(files)):
#print(image[0].info())
#print(image[0].header)
    print(image[i].header['FILENAME'],image[i].header['OBJECT'],image[i].header['EXPTIME'])

 The first two files are observations of 5s exposure in R band and the third is a longer exposure in a narrow filter. (Browse observation loogbook) 

### Display the images 
Let display the images with the same background and foreground limit values.

In [ ]:
sky_mean , std = [] , []
vmin = 10 
vmax = 1200
fig, axarr = plt.subplots(ncols=3, nrows=1, figsize=(14, 12))
for n in range(3):
    ax = axarr[n]
    ax.imshow(image[n].data, cmap='gray', origin='lower',vmin=vmin, vmax=vmax,norm=LogNorm())
    ax.text(200,100,image[n].header['FILENAME'],fontsize=15,color="w")
    ax.set_xlabel('X axis')
    mean_n,std_n = draw_rectangle(ax, image[n].data, 100, 500, 1500, 1900, color='w',text=True)
    print(n,int(mean_n),int(std_n))
    sky_mean.append(mean_n)
    std.append(std_n)
    ax.grid()
    

We should check using a star chart that HZ44 is the star in the middle of the frame.  
Look for the HZ44 in SIMBAD and ALADIN.

In [ ]:
## Aperture photometry with Photutils

In [ ]:
from photutils import DAOStarFinder
from photutils import find_peaks
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import aperture_photometry

from astropy.stats import sigma_clip, sigma_clipped_stats, mad_std

### Locating the stars

We can pass the position of the star to be measured but Photutils also includes a find_peaks() function to find local peaks in an image that are above a specified threshold value. Peaks are the local maxima above a specified threshold that are separated by a specified minimum number of pixels. 

Let´s use the first image 

In [ ]:
ximage = image[0].data
mean, median, std = sigma_clipped_stats(ximage, sigma=3.0)
threshold = median + (500. * std)
tbl = find_peaks(ximage, threshold, box_size=40)
tbl['peak_value'].info.format = '%.8g'  # for consistent table output
print(tbl[:10])  # print only the first 10 peaks

Displaying the peaks found with a circular red mark.

In [ ]:
fig = plt.figure(figsize=(9, 9))
radius = 30
vmin, vmax = 10, 1200
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=radius)
plt.imshow(ximage, cmap='gray', origin='lower',vmin=vmin, vmax=vmax , norm=LogNorm())
plt.grid()
apertures.plot(color='red', lw=1.5)

In [ ]:
phot_table = aperture_photometry(ximage, apertures)
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

The sources can be added if their positions are known or estimated.

In [ ]:
positions_new = [(560, 440), (1000, 750), (800, 1000)]

In [ ]:
fig = plt.figure(figsize=(9, 9))
radius = 30
vmin, vmax = 10, 1200
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=radius)
plt.imshow(ximage, cmap='gray', origin='lower',vmin=vmin, vmax=vmax , norm=LogNorm())
plt.grid()
apertures.plot(color='red', lw=1.5)

apertures_new = CircularAperture(positions_new, r=radius)
apertures_new.plot(color='yellow', lw=1.5)

We have selected two positions without any source to measure the sky background near HZ44.

In [ ]:
phot_table = aperture_photometry(ximage, apertures_new)
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

The background (last two positions) have values around 1.0E5 counts inside the aperture of radius 30 pixels.

### Global Background Subtraction
We can made a rougth estimate the background level using the median value for the whole image or better using a region without sources.

In [ ]:
median     = np.median(ximage)
median_sky = np.median(ximage[1000:1250,250:500])
print('median', median, ' sky ', median_sky)

In [ ]:
bkg = median_sky
phot_table = aperture_photometry(ximage - bkg, apertures) 
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

 Total flux   1022.0   964.0    1.467e+06  (counts) inside aperture with radius 30 pix   
 Net flux     1022.0   964.0    1.368e+06   after substracting the estimated background

### Measuring in several apertures 

It is posible to define several circular apertures and perform photometry in them.   
Let's use the HZ44 position.

In [ ]:
print(apertures)
HZ44_pos = positions[1]
print(HZ44_pos)

In [ ]:
radii = [10., 20., 30., 40.]
apertures = [CircularAperture(HZ44_pos, r=r) for r in radii]
phot_table = aperture_photometry(ximage, apertures)
for col in phot_table.colnames:
     phot_table[col].info.format = '%.5g'  # for consistent table output
print(phot_table)

## Local Background Subtraction
Classical aperture photometry uses a circular annulus around the star image to determine the background. We will select radius of 8 pixels for the aperture and a circular annulus of inner radius of 12 and outer radius of 20. 

In [ ]:
aperture = CircularAperture(HZ44_pos, r=30)
annulus_aperture = CircularAnnulus(HZ44_pos, r_in=50., r_out=70.)
plt.figure(figsize=(14,9))
plt.imshow(ximage, cmap='gray', origin='lower', vmin=vmin, vmax=vmax , norm=LogNorm())
aperture.plot(color='yellow', lw=1)
annulus_aperture.plot(color='red', lw=1)

In [ ]:
apers = [aperture, annulus_aperture]
phot_table = aperture_photometry(ximage, apers)
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

The last two columns show the total number of counts inside the cicle and the circular annulus. The areas of the apertures are different and we should correct before substracting the background by scaling the results for the same area

In [ ]:
print(aperture.area, annulus_aperture.area)

In [ ]:
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_sum  = bkg_mean * aperture.area
final_sum = phot_table['aperture_sum_0'] - bkg_sum
phot_table['residual_aperture_sum'] = final_sum
phot_table['residual_aperture_sum'].info.format = '%.8g'  # for consistent table output
print(phot_table['residual_aperture_sum'])  

In [ ]:
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

The last column shows the net counts obtained after substracting the scaled background

Let's repeat for the second file

In [ ]:
ximage = image[1].data
mean, median, std = sigma_clipped_stats(ximage, sigma=3.0)
threshold = median + (500. * std)
tbl = find_peaks(ximage, threshold, box_size=40)
tbl['peak_value'].info.format = '%.8g'  # for consistent table output
print(tbl[:10])  # print only the first 10 peaks

In [ ]:
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=radius)

In [ ]:
print(apertures)
HZ44_pos = positions[3]
print(HZ44_pos)

In [ ]:
aperture = CircularAperture(HZ44_pos, r=30)
annulus_aperture = CircularAnnulus(HZ44_pos, r_in=50., r_out=70.)
plt.figure(figsize=(9,9))
plt.imshow(ximage, cmap='gray', origin='lower', vmin=vmin, vmax=vmax , norm=LogNorm())
aperture.plot(color='yellow', lw=1)
annulus_aperture.plot(color='red', lw=1)

In [ ]:
apers = [aperture, annulus_aperture]
phot_table = aperture_photometry(ximage, apers)
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

In [ ]:
print(aperture.area, annulus_aperture.area)

In [ ]:
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_sum  = bkg_mean * aperture.area
final_sum = phot_table['aperture_sum_0'] - bkg_sum
phot_table['residual_aperture_sum'] = final_sum
phot_table['residual_aperture_sum'].info.format = '%.8g'  # for consistent table output
print(phot_table['residual_aperture_sum'])  

In [ ]:
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

This can be compared with our first HZ44 observation
 id xcenter ycenter aperture_sum_0 aperture_sum_1 residual_aperture_sum
      pix     pix                                                      
--- ------- ------- -------------- -------------- ---------------------
  1    1022     964      1466526.3      273523.19             1363955.1
  
fzt_ALrd120077.fits HZ44 R 5.0    1363955.1 c / 5s = 272791.02 c/s
fzt_ALrd120078.fits HZ44 R 5.0    1380058   c / 5s = 276011.6  c/s 